# Using Instrumental Variables for Treatment Effects in Quasi-Experiments

This is an example from Chapter 11 of the book [Methods Matter: Improving Causal Inference in Educational and Social Science](https://www.amazon.com/Methods-Matter-Improving-Inference-Educational-ebook/dp/B00HNSNBO4) by Richard Murnane and John Willett. This chapter explains how we can use Instrumental Variables Estimation to get a treatment effect using observational data.

In this example, we use IV to recover the treatment effect in cases where random assignment is applied to an offer to participate, where not every participates (compliers AND never takers allowed!), and where other people participate through some other means. Specifically, we would like to evaluate the effectiveness of a financial aid offer on the likelihood of a student finishing 8th grade in Bogotá, Colombia.

The data is sampled from [Vouchers for Private Schooling in Colombia: Evidence from a Randomized Natural Experiment](http://www.nber.org/papers/w8343) (2002) by Joshua Angrist and others.

Python implementation credit goes to J. Nathan Matias, who provides implementations of these methods in a [github repository](https://github.com/natematias/research_in_python).

In [1]:
import pandas as pd
import math
import statsmodels.formula.api as smf  # for doing statistical regression
import statsmodels.api as sm      # access to the wider statsmodels library, including R datasets

/Users/dereksnow/anaconda/envs/py36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Helper functions
def coefficient_to_odds_ratio(coefficient):
    """
    This function interprets a coefficient from a Logistic Regression model in as an odds ratio
    
    Given:
    coefficient -- coefficient from loistic regression output
    
    Returns:
    odds ratio (i.e. p/(1-p))
    """
    odds = math.exp(coefficient)
    print("Those using financial aid are {:2f} times more likely to finish the 8th grade!".format(odds))
    
def ate_odds_ratio(prob0, prob1):
    """
    This function calculates the average treatment effects as an odds ratio
    when the outcome is binary.
    
    Given:
    prob0 -- probability given no treatment
    prob1 -- probability given treatment
    
    Returns:
    odds ratio (i.e. p/(1-p))
    """
    odds = (prob1/(1-prob1)) - (prob0/(1-prob0))
    print("Odds ratio: {}".format(odds))

## Importing the Dataset

The IV is the introduced lottry, a small subgroup that did actually undergo a randomised experiment. Then this information can be use to extend it to students who received financial aid in a different way to enure that the receiving of money is what is important and not the reason to have money. Again here, the IV is a random "observation" experiment closer or at the same proximity as the treatment without the entangled confounds. 

The dataset includes the following variables:
- `finish8th`: did the student finish 8th grade or not (outcome variable)
- `won_lottry`: won the lottery to receive offer of financial aid
- `use_fin_aid`: did the student use financial aid of any kind (not exclusive to the lottery) or not
- `base_age`: student age
- `male`: is the student male or not

In [3]:
# Import data set
voucher_df = pd.read_sas('colvoucher.sas7bdat') # reading in sas file
voucher_df.head()

,id,won_lottry,male,base_age,finish8th,use_fin_aid
0,3.0,1.0,0.0,11.0,1.0,1.0
1,4.0,0.0,1.0,11.0,1.0,1.0
2,5.0,0.0,1.0,11.0,1.0,0.0
3,6.0,0.0,0.0,9.0,0.0,0.0
4,10.0,1.0,1.0,11.0,1.0,1.0


## Summary Statistics

In [4]:
print("==============================================================================")
print("                              OVERALL SUMMARY"                                 )
print("==============================================================================")

print(voucher_df.describe())

for i in range(2):
    print("==============================================================================")
    print("                         USE FINANCIAL AID = %(i)d" % {"i":i}                  )
    print("==============================================================================")
    print(voucher_df[voucher_df['use_fin_aid']==i].describe())

                              OVERALL SUMMARY
                id   won_lottry         male     base_age    finish8th  \
count  1171.000000  1171.000000  1171.000000  1171.000000  1171.000000   
mean   1357.010248     0.505551     0.504697    12.004270     0.681469   
std     890.711584     0.500183     0.500192     1.347038     0.466106   
min       3.000000     0.000000     0.000000     7.000000     0.000000   
25%     616.000000     0.000000     0.000000    11.000000     0.000000   
50%    1280.000000     1.000000     1.000000    12.000000     1.000000   
75%    1982.500000     1.000000     1.000000    13.000000     1.000000   
max    4030.000000     1.000000     1.000000    17.000000     1.000000   

       use_fin_aid  
count  1171.000000  
mean      0.581554  
std       0.493515  
min       0.000000  
25%       0.000000  
50%       1.000000  
75%       1.000000  
max       1.000000  
                         USE FINANCIAL AID = 0
                id  won_lottry        male    base_

## Average Treatment Effect

We can calculate the ATE empirically simply using the summary stats above, however this is an unconditional average treatment effect.

Use the fact that 

$ ATE = E[Y=1|T=1] - E[Y=1| T=0] $.

In case of binary outcome, we use

$ ATE = Pr[Y=1|T=1] - Pr[Y=1|T=0] $

and subsitute the conditional expectations/probabilities with their empirical estimates.

In [5]:
# Calculate unconditional average treatment effect
ate = 0.737151 - 0.604082
ate

0.133069

In [7]:


# Calculate this in terms of odds ratio
ate_odds_ratio(0.604082, 0.737151)

## Odds ratio says how many times more likley they are to pass

Odds ratio: 1.2786901424379509


## A Naive Logistic Regression

In [8]:
## Generalized Linear Model 

print("==============================================================================")
print("                            LOGISTIC REGRESSION"                               )
print("==============================================================================")
result = smf.glm(formula = " finish8th ~ use_fin_aid + male + base_age", 
                 data=voucher_df,
                 family=sm.families.Binomial()).fit()
print(result.summary())

                            LOGISTIC REGRESSION
                 Generalized Linear Model Regression Results                  
Dep. Variable:              finish8th   No. Observations:                 1171
Model:                            GLM   Df Residuals:                     1167
Model Family:                Binomial   Df Model:                            3
Link Function:                  logit   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -694.99
Date:                Mon, 16 Jul 2018   Deviance:                       1390.0
Time:                        20:18:55   Pearson chi2:                 1.17e+03
No. Iterations:                     4                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       4.2496      0.596      7.128      0.000       3.081       5.418
u

In [9]:
# Log odd units to odds ratio units
# So the conditional controls in the logistic regression increase
# The ATE from the unconditional amount. 
coefficient_to_odds_ratio(result.params[1])

Those using financial aid are 1.781220 times more likely to finish the 8th grade!


## Two Stage Least Squares Estimation

The method used for instrumental variables. 

Recall that using 2SLS estimation for the instrumental variables model requires estimating two equations:

__Stage 1:__ 

$ T = Z\alpha + X\psi + \nu $

__Stage 2:__

$ Y = {\hat{T}} \gamma + X\beta + \epsilon $

The instrument we choose must satisfy the two conditions:
 1. __Instrument relevance:__ $ Cov(Z, T) \neq 0 $
 2. __Instrument exogeneity:__ $ Cov(Z, \epsilon) = 0 $
 
Why is there endogeneity and what instrument is available in our dataset to combat this?

In [10]:
## The first step is important to identify the strenght of the ...
## because not everyone that won the lottery for their studies 
## ..used it. which si weird, but yea.

print("==============================================================================")
print("                                  FIRST STAGE"                                 )
print("==============================================================================")
result = smf.glm(formula = "use_fin_aid ~ won_lottry + male + base_age", 
                 data=voucher_df,
                 family=sm.families.Binomial()).fit()
voucher_df['use_fin_aid_fitted']= result.predict()
print(result.summary())

print()
print()
print("==============================================================================")
print("                                  SECOND STAGE"                                )
print("==============================================================================")
result = smf.glm(formula = " finish8th ~ use_fin_aid_fitted + male + base_age", 
                 data=voucher_df,
                 family=sm.families.Binomial()).fit()
print(result.summary())

                                  FIRST STAGE
                 Generalized Linear Model Regression Results                  
Dep. Variable:            use_fin_aid   No. Observations:                 1171
Model:                            GLM   Df Residuals:                     1167
Model Family:                Binomial   Df Model:                            3
Link Function:                  logit   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -488.00
Date:                Mon, 16 Jul 2018   Deviance:                       975.99
Time:                        20:27:01   Pearson chi2:                 1.16e+03
No. Iterations:                     5                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3455      0.731      0.472      0.637      -1.088       1.779
won_lo

In [12]:
result.params[1]

0.7742597028704997

In [13]:
# Transforming the log-odds units into odds ratio ,,
# Because you are using logistic regression. 
coefficient_to_odds_ratio(result.params[1])

Those using financial aid are 2.168986 times more likely to finish the 8th grade!


## Interpreting the Local Average Treatment Effect

When we use IV to get a causal effect, what are our results actually telling us? According to Murnane and Willett, "an estimate of a treatment effect obtained by IV methods should be regarded as an __estimated local average treatment effect (LATE)__. As mentioned earlier in the lecture slides, the LATE estimate can depend on your choice of instruments.

As stated in the book:

- Compliers "are willing to have their behavior determined by the outcomes of the lottery, regardless of the particular experimental conditions to which they were assigned" (278).
- Always-Takers "are families who will find and make use of financial aid to pay private-school fees" regardless of the lottery. They may find aid outside the lottery
- Never-takers are the mirror image of always-takers: "they will not make use of financial aid to pay childrens' fees at a private secondary school under any circumstances" (278)
- (there are other possible groups, like "defiers" (Gennetian et all, 2005) who always do the opposite of what investigators ask them to do, but we make the assumption of "no defiers" in this dataset)

In this context, IV estimates of the __local average treatment effect (LATE) for this quasi-experiment only applies to "compliers"__--and not to never-takers or always-takers.

__Tests for instrument validity/strength of instruments is an established line of research that we can point you to if wanted!__